# Assignment on Random Search, Hyperopt

**Import required libraries**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

**Read the dataset into python environment**

In [2]:
df=pd.read_csv('/content/titanic.csv')

Check basic details of dataset

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
df.shape

(891, 12)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
#Check for null values
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

**Split the data set into test and train**

In [7]:
X = df[['Pclass','SibSp','Parch','Fare']]
y = df.Survived

In [8]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.25)

**Standard Scaling**

In [9]:
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.fit_transform(X_test)

**Apply SVM**

In [11]:
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report
model1 = SVC(kernel='linear', C=1, gamma='auto')
model1.fit(X_train,y_train)
y_pred=model1.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.74      0.86      0.79       139
           1       0.68      0.49      0.57        84

    accuracy                           0.72       223
   macro avg       0.71      0.68      0.68       223
weighted avg       0.72      0.72      0.71       223



**Apply 5-Fold Cross Validation to SVM**

In [13]:
from sklearn.model_selection import cross_val_score, KFold
model = SVC()
kf=KFold(n_splits=5)
score=cross_val_score(model,X,y,cv=kf)
print("Cross Validation Scores are {}".format(score))
print("Average Cross Validation score :{}".format(score.mean()))

Cross Validation Scores are [0.62011173 0.66853933 0.66853933 0.66292135 0.70786517]
Average Cross Validation score :0.6655953800765803


Apply random search to SVM to identify the best possible model

In [17]:
parameters ={'C': [0.1, 1, 10, 100, 1000],
             'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
             'kernel': ['rbf']}

In [18]:
from sklearn.model_selection import RandomizedSearchCV
random_search=RandomizedSearchCV(estimator = SVC(), param_distributions=parameters,verbose=1, n_jobs=-1,n_iter=50)
random_result = random_search.fit(X_train, y_train)
print('Best Score: ', random_result.best_score_*100)
print('Best Params: ', random_result.best_params_)

Fitting 5 folds for each of 25 candidates, totalling 125 fits
Best Score:  70.36247334754798
Best Params:  {'kernel': 'rbf', 'gamma': 0.01, 'C': 1000}


**Use hyperopt**

In [19]:
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials

In [21]:
space = {    
    "C": hp.choice("C", [0.1, 1, 10, 100, 1000]),
    "gamma": hp.choice("gamma", [1, 0.1, 0.01, 0.001, 0.0001]),
    "kernel": hp.choice("kernel",['rbf'])
}

In [22]:
# define objective function

def hyperparameter_tuning(params):
    clf = SVC(**params)
    acc = cross_val_score(clf, X, y,scoring="accuracy").mean()
    return {"loss": -acc, "status": STATUS_OK}

In [23]:
# Initialize trials object
trials = Trials()

best = fmin(
    fn=hyperparameter_tuning,
    space = space, 
    algo=tpe.suggest, 
    max_evals=50, 
    trials=trials
)

print("Best: {}".format(best))

100%|██████████| 50/50 [00:59<00:00,  1.19s/it, best loss: -0.7060134329295085]
Best: {'C': 4, 'gamma': 1, 'kernel': 0}


Best params - C -1000, gamma - 0.1

In [24]:
#Create a model with best parameters
model = SVC(C=1000, gamma=0.01,kernel='rbf')

model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.76      0.91      0.83       139
           1       0.78      0.54      0.63        84

    accuracy                           0.77       223
   macro avg       0.77      0.72      0.73       223
weighted avg       0.77      0.77      0.76       223

